# AM207 Final Project

Paper: *Learned Uncertainty-Aware (LUNA) Bases for Bayesian Regression using Multi-Headed Auxiliary Networks*

In [ ]:
# this code is for an elaine problem:
import sys
sys.path.append('/Users/elainecunha/opt/anaconda3/envs/py3.8/lib/python3.8/site-packages')

In [ ]:
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import autograd.numpy.random as npr
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import sys
import time

# our libraries
import utils
from nlm import NLM
from feed_forward import Feedforward
import bayes_helpers as bh

from config import *

### Generate Data

In [ ]:
# training data
x_train, y_train, x_test = utils.generate_data(training_seed)

# for model evaluation: computing log likelihood
x_valid, y_valid, x_test_not_used = utils.generate_data(valid_seed)

### Train NLM

In [ ]:
opt_params["max_iteration"] = real_max_iteration
# train NLMs
reg_params = [0.01, 0.1, 1.0,10]
nlms = []
log_ls = []

for i,r in enumerate(reg_params):

    # append model to list of NLMs
    nlms.append(NLM(prior_variance,y_noise_variance, r, nlm_architecture, random_seed))
    
    # train model
    t0 = time.time()
    nlms[i].train(x_train, y_train, opt_params)
    nlm_time = np.round(time.time() - t0, 3)
    log_ls.append(nlms[i].get_log_l(x_train,y_train,x_valid,y_valid))
    
    print('Model {}: {} seconds'.format(i, nlm_time))

### Calculate Posterior and Prior Predictives

In [ ]:
# calculate posterior and prior predictives
post_pred_mean = []
post_pred_upper = []
post_pred_lower = []
prior_predictive_samples = []

for model in nlms:
    
    # get posterior and prior samples
    post_pred, post_pred_samples = model.predict(x_test, prior=False)
    prior_pred, prior_pred_samples = model.predict(x_test, prior=True)

    # calculate posterior predictive mean
    pp_mean = np.mean(post_pred_samples, axis=0)
    pp_upper = np.percentile(post_pred_samples, 97.5, axis=0)
    pp_lower = np.percentile(post_pred_samples, 2.5, axis=0)

    # store results
    post_pred_mean.append(pp_mean)
    post_pred_upper.append(pp_upper)
    post_pred_lower.append(pp_lower)
    prior_predictive_samples.append(prior_pred_samples)

### Plot Results

In [ ]:
# calculate ground truth line
f = lambda x: (x**3)
x_true = np.linspace(-10, 10, 100)
y_true = f(x_true)

In [ ]:
# plot

num_figures = len(nlms)

fig,ax = plt.subplots(num_figures, 2, figsize=(10,20))

for i,model in enumerate(nlms):
    ax[i][0].scatter(x_train, y_train, color='red')
    ax[i][1].scatter(x_train, y_train, color='red')
    ax[i][0].plot(x_true, y_true, color='black')
    ax[i][1].plot(x_true, y_true, color='black')
    ax[i][1].plot(x_test.flatten(), post_pred_mean[i], color='blue')
    ax[i][1].fill_between(x_test.flatten(), post_pred_upper[i], post_pred_lower[i], color='blue', alpha=0.1, label='95% Pred. Interval') 

    for j in range(prior_predictive_samples[i].shape[1]):
        
        ax[i][0].plot(x_test.flatten(), prior_predictive_samples[i][j,:], alpha = 0.1, color='blue')

    ax[i][0].axis([-5, 5, -150, 150])
    ax[i][1].axis([-5, 5, -150, 150])
    ax[i][0].set_title('Prior Predictive, reg_param={}'.format(reg_params[i]))
    ax[i][1].set_title(f'Posterior Predictive, reg_param={reg_params[i]} \n Log Likelihood {round(log_ls[i],3)}')

plt.show()